# Create data files for baseline models

The purpose of this single-use notebook is to create dummy `data0.csv` files that can be used to create finetuning datasets using `create_finetuning_dataset.py`

In [ ]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict

In [ ]:
from evals.locations import DATASET_DIR, EXP_DIR

In [ ]:
STUDY_NAME = "BASELINE_MODEL"
MODEL = "baseline_model" # dummy model config

In [ ]:
SEED = 0
np.random.seed(SEED)

## Helper functions

In [ ]:
def save_data0_df(out_dict, dataset_name, overwrite=True):
    out_dict['complete'] = True
    df = pd.DataFrame(out_dict)
    df['response'] = df['identity']
    df['logprobs'] = ['[{}]'] * len(df)
    # check that the directory exists
    (EXP_DIR / STUDY_NAME / f"object_level_{dataset_name}").mkdir(parents=True, exist_ok=True)
    # does the file already exist?
    if (EXP_DIR / STUDY_NAME / f"object_level_{dataset_name}" / f"data{SEED}.csv").exists():
        if not overwrite:
            print(f"File already exists, not overwriting: {EXP_DIR / STUDY_NAME / f'object_level_{dataset_name}' / f'data{SEED}.csv'}")
            return
    df.to_csv(EXP_DIR / STUDY_NAME / f"object_level_{dataset_name}" / f"data{SEED}.csv")

In [ ]:
def create_and_save_df(name, response_properties, overwrite=False):
    assert 'identity' in response_properties, 'identity must be a response property'
    path_to_jsonl = DATASET_DIR / f'train_{name}.jsonl'
    with open(path_to_jsonl, 'r') as f:
        lines = f.readlines()
    
    out = {rp: [] for rp in response_properties}
    out['string'] = []

    for line in lines:
        data = json.loads(line)
        out['string'].append(data['string'])
        for response_property, function in response_properties.items():
            out[response_property].append(function(data))
    
    save_data0_df(out, name, overwrite=overwrite)
    create_hydra_config(name, response_properties)
    print(f"Saved {len(out['string'])} rows for {name}")


In [ ]:
def create_hydra_config(task, response_property):
    path_to_config = EXP_DIR / STUDY_NAME / f"object_level_{task}" / "logs"/ "date" / "time" / ".hydra" / "config.yaml"
    path_to_config.parent.mkdir(parents=True, exist_ok=True)
    config = f"""study_name: {STUDY_NAME}
study_dir: {EXP_DIR / STUDY_NAME}
exp_dir: {EXP_DIR / STUDY_NAME}/object_level_{task}
note: ''
seed: 0
limit: 500
n_samples: 1
strings_path: null
filter_strings_path: null
reset: false
logging: INFO
print_prompt_and_response: false
cache_dir: ~
prompt_history_dir: ~
language_model:
  temperature: 0.0
  logprobs: 0
  model: {MODEL}
  top_p: 1.0
  max_tokens: null
  num_candidates_per_completion: 1
  insufficient_valids_behaviour: error
task:
  num: null
  set: train
  name: {task}
  dataset_path: evals/datasets/train_${task}.jsonl
  filter_strings_path: null
  shuffle: true
  exclusion_rule_groups:
  - default
organization: DEFAULT_ORG
anthropic_tag: ANTHROPIC_API_KEY
openai_tag: OPENAI_API_KEY
anthropic_num_threads: 12
openai_fraction_rate_limit: 0.9
prompt:
  method: object_level/minimal
  messages:
  - role: system
    content: ''
  - role: user
    content: ''
    """
    with open(path_to_config, "w") as f:
        f.write(config)
    

`response_properties` is a dict with a response property as key and a function that takes in a parsed line from the .jsonl and return the value of the response property.

## `number_triplets`

In [ ]:
def random_number(_):
    return f"{np.random.randint(100, 999)}"

In [ ]:
def random_true_false(_):
    return np.random.choice(['true', 'false'])

In [ ]:
def random_digit(_):
    return str(np.random.randint(0, 9))

In [ ]:
response_properties = {
    "identity": random_number,
    "is_even": random_true_false,
    "first_character": random_digit,
    "last_character": random_digit,
}

In [ ]:
create_and_save_df('number_triplets', response_properties)

## `wikipedia`

In [ ]:
def target(data):
    return data['target']

In [ ]:
def random_character(_):
    return np.random.choice(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])

In [ ]:
response_properties = {
    "identity": target,
    "syllable_count": random_digit,
    "first_character": random_character,
    "last_character": random_character,
}

In [ ]:
create_and_save_df('wikipedia', response_properties)

## `daily_dialog`

In [ ]:
response_properties = {
    "identity": target,
    "syllable_count": random_digit,
    "first_character": random_character,
    "last_character": random_character,
}

In [ ]:
create_and_save_df('daily_dialog', response_properties)

## `personal_preferences`
This will just fill in dummies, and we have to hand write them in the generated file

In [ ]:
def dummy(_):
    return "XXX"

In [ ]:
response_properties = {
    "identity": dummy,
    "syllable_count": random_digit,
    "first_character": random_character,
    "last_character": random_character,
}

In [ ]:
create_and_save_df('personal_preferences', response_properties)

_**Replace the dummy values by hand**_

## `self_referential`
This will just fill in dummies, and we have to hand write them in the generated file

In [ ]:
response_properties = {
    "identity": dummy,
    "syllable_count": random_digit,
    "first_character": random_character,
    "last_character": random_character,
}

In [ ]:
create_and_save_df('self_referential', response_properties)

_**Replace the dummy values by hand**_

## `writing_stories`
Here, we will just fill in the lorem ipsum text

In [ ]:
LOREM_IPSUM = [
    "Nullam tincidunt porta ligula sit amet scelerisque. Quisque dapibus nulla in tincidunt vehicula. Aliquam elit neque, vulputate a lectus a, vehicula gravida leo. Aenean eget tempor lacus. Cras pretium vitae tortor ut volutpat. Sed elementum, purus semper convallis mattis, libero est vestibulum leo, eu porttitor nibh nunc ac eros. Cras nisl lacus, ornare eu turpis id, faucibus lobortis justo. Ut a sagittis justo. Nunc pharetra felis eros, et semper nulla mattis sed. Nunc accumsan velit quis suscipit gravida. Vestibulum vulputate velit quis libero dictum euismod. Pellentesque urna purus, feugiat sit amet ex sodales, blandit fermentum ex. Fusce convallis arcu lacus, et maximus neque dapibus non. Proin mattis, dui quis luctus dictum, lorem lectus commodo metus, id suscipit libero metus in nisi. Donec et venenatis lacus.",
"Integer ac imperdiet sem. Mauris eget accumsan leo. Curabitur pharetra, magna sed dapibus volutpat, mi enim blandit dolor, eu tristique velit felis vitae velit. Praesent eros arcu, ultricies vel massa sit amet, consequat mollis nulla. Cras imperdiet lorem eget diam gravida efficitur. Praesent interdum cursus eros. Praesent odio augue, vestibulum non elementum ac, vehicula feugiat sem. Curabitur vestibulum rutrum enim, ac volutpat nisl malesuada sit amet. Donec sed lorem eleifend orci placerat pellentesque.",
"Praesent finibus erat id nisl tempor, non fringilla purus efficitur. Phasellus a posuere arcu. Mauris vel vehicula justo. Aenean ut ex nec sem interdum vulputate eget ut metus. Aliquam sagittis urna nec rutrum tempus. Nulla facilisi. Suspendisse aliquam risus in purus dictum ultricies. Maecenas quis nisi ullamcorper, commodo risus vel, ornare tortor. Praesent varius ultrices aliquam. Sed volutpat aliquam massa ut interdum. Ut turpis arcu, pharetra eu neque eu, condimentum pharetra dolor. Interdum et malesuada fames ac ante ipsum primis in faucibus. Fusce facilisis, elit non tempus pharetra, velit elit vulputate diam, eget porttitor turpis sapien non ipsum.",
"Ut faucibus egestas lorem ut pellentesque. Suspendisse vestibulum ante quis risus pellentesque convallis. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Sed fringilla magna risus, ut accumsan justo aliquam nec. Mauris non mattis ipsum. Donec nec fringilla erat. Aliquam tempor non arcu sollicitudin euismod. Aenean consequat varius iaculis.",
"Fusce porta ultricies enim ac fringilla. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque tincidunt non tortor et imperdiet. Interdum et malesuada fames ac ante ipsum primis in faucibus. Praesent condimentum imperdiet erat, quis varius libero. Nulla facilisi. Quisque at nibh risus.",
"Fusce justo mi, tincidunt id auctor at, elementum a leo. In ac diam dui. Vivamus semper vitae libero eget iaculis. Nullam dui felis, elementum a vestibulum convallis, consectetur sed metus. Vestibulum non sollicitudin mauris. Duis mattis ipsum vitae rutrum facilisis. Quisque sagittis lacinia nisl eu aliquet. Pellentesque ante mauris, maximus nec lobortis quis, ultricies lacinia odio. Maecenas ac ullamcorper metus. Aliquam sodales mauris elit, auctor porta nisi tristique ac. Nulla ullamcorper, ante ac mollis tincidunt, nibh orci placerat augue, a dictum elit sem vitae lacus. Donec pellentesque tortor quis nunc rhoncus viverra. Aliquam ex nunc, sollicitudin sed odio eu, tincidunt vulputate arcu. Suspendisse potenti. Ut id cursus tortor. Quisque odio tellus, ullamcorper eu enim a, sodales eleifend metus.",
"Cras malesuada tincidunt justo, in facilisis turpis interdum sit amet. Nulla condimentum lacus in sapien bibendum hendrerit. Nunc sollicitudin sed neque sed convallis. Fusce faucibus consequat porta. Maecenas quis est sit amet lorem pharetra condimentum id nec magna. Vestibulum tellus enim, mattis eu commodo in, blandit a orci. Aliquam pellentesque, sapien in maximus volutpat, elit tortor tristique tortor, et accumsan eros felis et tortor. Vivamus in pharetra tortor. Proin venenatis nisi non lacus gravida ullamcorper. Duis commodo scelerisque erat sed semper. Vestibulum viverra, velit sit amet fermentum elementum, tortor sem blandit orci, quis sollicitudin odio justo non lacus. Mauris venenatis dapibus libero, eu ornare odio rutrum ut. Fusce dictum porta mollis. Suspendisse lectus risus, ornare et commodo nec, hendrerit quis leo.",
"Quisque elementum iaculis nibh, et aliquam orci feugiat at. Ut venenatis semper dui at bibendum. Mauris tincidunt, felis ut mattis facilisis, felis risus egestas turpis, eget suscipit ligula tortor id risus. Maecenas id augue massa. Nulla velit turpis, facilisis eget mattis fermentum, imperdiet quis mi. Morbi sed dapibus lorem. Maecenas euismod euismod orci sed suscipit. Duis ultricies ut felis nec rutrum. Sed est eros, vulputate vel bibendum eget, consequat nec magna. Fusce libero enim, bibendum id pretium et, malesuada nec ex. Interdum et malesuada fames ac ante ipsum primis in faucibus. Suspendisse auctor imperdiet quam, id fringilla neque pharetra vitae. In egestas et lectus id gravida. Donec risus lectus, pretium sed felis at, hendrerit rutrum lorem. Nullam vitae tortor blandit, vulputate diam in, efficitur ipsum.",
"Duis dictum suscipit laoreet. Vivamus euismod urna in sem lacinia tempus. Vestibulum eget libero ornare, consectetur libero ut, accumsan justo. Aenean id dignissim elit. Integer facilisis lorem eu ante volutpat, ac mattis tortor sagittis. Maecenas mattis scelerisque nisi, congue tincidunt augue hendrerit nec. Pellentesque non lectus non arcu rhoncus pharetra a non risus. Maecenas odio enim, tincidunt eget sapien non, blandit suscipit massa.",
"Curabitur at quam eget mauris imperdiet congue quis eget nibh. Mauris vel lacus at erat dictum luctus ac nec urna. Sed a blandit tortor, et porttitor turpis. Donec vulputate volutpat luctus. Nam fringilla nulla nisi, vitae imperdiet sapien faucibus eget. Integer facilisis tempus ante. Mauris scelerisque nulla sit amet ante porttitor, quis mollis tellus maximus. Aliquam tristique tempus placerat. Phasellus tellus risus, vulputate quis hendrerit nec, luctus sit amet metus. Suspendisse pulvinar vel tellus porta efficitur. Donec id tincidunt libero, vel egestas metus. Cras est purus, auctor in turpis et, auctor bibendum massa. Quisque vel consectetur quam. Nunc volutpat id sapien eget congue.",
"Nam rhoncus eros a leo elementum dapibus. Ut sit amet lacus at turpis tempus hendrerit at sit amet est. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia curae; Pellentesque molestie lorem ipsum, dictum vestibulum nisl ultricies vel. Fusce imperdiet nisi eros, ultricies rhoncus sapien lobortis ut. Etiam scelerisque dictum quam vel mollis. Nulla odio lacus, rhoncus quis orci et, pharetra rhoncus erat.",
"Sed vel orci eu sapien varius viverra. Nam facilisis sit amet sapien bibendum blandit. Nunc vestibulum, tellus et scelerisque laoreet, quam risus rutrum turpis, vel eleifend nisl sem non velit. Nunc egestas quis turpis eget commodo. Aliquam erat volutpat. Ut id mattis lectus. Mauris facilisis lobortis porta. Sed eget purus semper, gravida elit semper, convallis enim. Sed et finibus velit. Proin porta libero mollis, malesuada nulla sed, posuere sem. Suspendisse congue placerat quam, a egestas lorem commodo facilisis. Vivamus euismod interdum ipsum, sed pulvinar mi sollicitudin a. Aliquam eleifend scelerisque euismod. Proin in libero quis ex rhoncus ornare ac a lorem. Vivamus ac faucibus purus. Sed et elit faucibus, cursus lorem quis, commodo sapien.",
"Donec pretium blandit urna nec rutrum. Mauris et dolor in purus maximus iaculis eget ac sapien. Vestibulum justo velit, congue at consequat sed, dapibus sed mi. Quisque convallis odio non lorem ultricies euismod. Nunc tempor erat magna, nec tristique ipsum gravida varius. Maecenas nec nisl dui. Nam pellentesque, tellus vitae pretium sagittis, lacus dolor rhoncus est, eget blandit odio diam id mi.",
"Quisque at malesuada tortor, id imperdiet eros. Vivamus feugiat lectus mauris, quis sagittis felis suscipit a. Phasellus magna tellus, dictum id dictum vitae, fermentum nec urna. Vestibulum tristique, dolor in sollicitudin feugiat, tortor lorem mattis sem, vel tempus nisl nisi in ante. Morbi rhoncus lorem nec magna interdum dictum. Cras aliquam libero sed imperdiet sagittis. Sed non neque arcu. Proin ut massa non metus congue lobortis vitae vitae odio. In hac habitasse platea dictumst. Suspendisse condimentum mattis arcu quis pharetra. Morbi maximus magna augue, eu fringilla ex convallis a. Ut iaculis tellus enim, quis porta erat finibus ut. Praesent id metus sagittis, venenatis velit a, porttitor neque. Aliquam luctus laoreet est vehicula rhoncus. Pellentesque id porttitor elit, at convallis metus. Aliquam ipsum nisl, pretium nec sagittis quis, porta eget purus.",
"Vestibulum vitae orci vulputate, pretium elit non, dictum eros. In venenatis, nulla eu bibendum laoreet, arcu mauris mattis lacus, eu luctus tellus lorem non augue. Ut at urna sed orci semper eleifend. Mauris blandit sagittis ex ac tincidunt. Pellentesque feugiat quam nunc, sit amet condimentum dolor imperdiet vulputate. Sed ipsum ipsum, varius in sem sed, semper blandit ligula. Ut pretium pretium tincidunt.",
"Aliquam in porta metus, et porttitor lectus. Etiam malesuada fermentum egestas. Curabitur gravida blandit suscipit. Sed sodales, ante eu posuere efficitur, mauris nunc convallis augue, nec ullamcorper eros libero vel nunc. Nulla facilisi. Integer aliquam eleifend orci eget ultricies. Morbi maximus augue in mollis egestas. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia curae; Maecenas varius arcu et enim malesuada consectetur. Nam luctus ligula non risus imperdiet bibendum. Donec mi sem, sollicitudin at justo ut, volutpat luctus arcu. Curabitur ut elit a erat dictum eleifend ac ut erat. Sed nunc libero, ultricies in vehicula ac, faucibus non lorem. Aliquam erat volutpat.",
"Phasellus imperdiet elit nec rutrum condimentum. Ut vestibulum, justo eget imperdiet posuere, mi nisl feugiat lorem, eget sodales ex arcu eget quam. Interdum et malesuada fames ac ante ipsum primis in faucibus. Aenean luctus euismod velit a tempus. Mauris quis ultricies arcu. Suspendisse potenti. In et placerat erat. Duis vitae efficitur ligula, ut ullamcorper velit. Suspendisse potenti. Morbi quis lobortis tortor. Suspendisse potenti.",
"Quisque consectetur, tellus quis rhoncus volutpat, massa erat interdum nisl, nec semper dolor risus sit amet mauris. Vivamus nec elit augue. Maecenas tristique feugiat urna. Pellentesque efficitur maximus vehicula. Nullam ut aliquet dolor, sed rhoncus nisi. Nunc sodales convallis nisl, ut sollicitudin mi condimentum sed. Morbi vitae dui et augue ultrices aliquet. Curabitur id nisi hendrerit, cursus leo quis, blandit dui. Ut mi ex, euismod non pharetra in, feugiat sed dolor. Nunc laoreet odio non ex hendrerit, vitae posuere risus ornare.",
"Donec commodo neque eu maximus mollis. Sed quis iaculis orci, eu aliquam dolor. Suspendisse dapibus, felis vel dignissim maximus, ante felis dictum libero, vel vestibulum tellus lorem at neque. Nam sollicitudin dignissim mauris, et interdum diam semper et. Sed mollis tortor nec lacus sagittis, sed ultrices justo elementum. Integer ornare varius ipsum, eu bibendum eros molestie at. In odio ipsum, lacinia quis leo a, congue commodo lectus. Mauris tincidunt orci vel odio laoreet, nec eleifend quam mattis. In eget aliquet sem.",
"Quisque feugiat suscipit tempor. Aenean cursus augue nec orci lacinia aliquam. Quisque eu semper nibh. Integer ac consectetur urna, a pretium tortor. Aenean fringilla faucibus erat, sit amet aliquet turpis vehicula at. Vestibulum rutrum elit sit amet leo congue, nec imperdiet nunc placerat. Donec quis maximus arcu. In bibendum posuere vehicula.",
]

In [ ]:
import nltk
nltk.download('names')
nltk.download('words')

In [ ]:
from nltk.corpus import words
from nltk.corpus import names
ALL_WORDS = words.words()
ALL_NAMES = names.words()
print(f"Loaded {len(ALL_WORDS)} words from nltk")
print(f"Loaded {len(ALL_NAMES)} names from nltk")

In [ ]:
def two_lorem_ipsum_paragraphs(_):
    return np.random.choice(LOREM_IPSUM) + "\n" + np.random.choice(LOREM_IPSUM)

In [ ]:
def random_word(_):
    return np.random.choice(ALL_WORDS)

In [ ]:
def random_name(_):
    return np.random.choice(ALL_NAMES)

In [ ]:
response_properties = {
    "identity": two_lorem_ipsum_paragraphs,
    "first_word": random_word,
    "writing_stories/main_character_name": random_name,
}

In [ ]:
create_and_save_df('writing_stories', response_properties)

## `dear_abbie`
I tried using the raw dataset from https://data.world/the-pudding/dear-abby to get ground truth. This does not work: the dataset does not provide the ground truth answer. Half are on the website and could be scraped. 

Instead, I'll use random data.

This will take a long time.

In [ ]:
def random_sentiment(_):
    return np.random.choice(['positive', 'negative'])

In [ ]:
response_properties = {
    "identity": two_lorem_ipsum_paragraphs,
    "sentiment": random_sentiment,
    "first_word": random_word,}

In [ ]:
create_and_save_df('dear_abbie', response_properties)

## Create finetuning dataset
Put in the task definitions here to create the finetuning data files

In [ ]:
from evals.utils import run_command

In [ ]:
tasks = {
    "wikipedia": ["identity", "syllable_count", "first_character", "last_character"],
  	"dear_abbie": ["identity", "sentiment"],
 	"number_triplets": ["identity", "is_even", "last_character", "first_character"],
 	"daily_dialog": ["identity", "syllable_count", "first_character", "last_character"],
  	"personal_preferences": ["identity", "syllable_count", "first_character", "last_character"],
  	"self_referential": ["identity", "syllable_count", "first_character", "last_character"],
  	"writing_stories": ["identity", "first_word", "writing_stories/main_character_name"]}

In [ ]:
PROMPT = "meta_level/minimal"

In [ ]:
for task, response_properties in tasks.items():
    for response_property in response_properties:
        dir = EXP_DIR / STUDY_NAME / f"object_level_{task}"
        command = f"python -m evals.create_finetuning_dataset_configs --study_name {STUDY_NAME} --model_config {MODEL} --task_config {task} --response_property_config {response_property} --train_base_dir {dir} --val_base_dir {dir} --prompt_config {PROMPT}"
        print(f"📀 Creating dataset for {task}, {response_property}: {command}")
        run_command(command)
        print(f"📀 Done creating dataset for {task}, {response_property}")

Create the actual finetuning dataset

In [ ]:
command = f"python -m evals.create_finetuning_dataset study_name={STUDY_NAME} dataset_folder={MODEL}"
print(f"📀 Creating dataset for all tasks: {command}")
run_command(command)

Now, you can finetune a baseline model with\
`python -m evals.run_finetuning study_name=BASELINE_MODEL/baseline_model notes=baseline`

Make sure to copy the finetuning hyperparameters.